# Лабораторная работа №2


_ПИН-212 Жан-Лин Никита_

In [28]:
# Импортируем необходимые библиотеки

# Pandas используется для работы с данными в табличном формате
import pandas as pd

# Импортируем функции для разделения данных на обучающую и тестовую выборки, а также для поиска гиперпараметров модели
from sklearn.model_selection import train_test_split, GridSearchCV

# Импортируем классы стандартизации признаков
from sklearn.preprocessing import StandardScaler

# Импортируем функции для оценки качества модели
from sklearn.metrics import classification_report, accuracy_score

# Импортируем классы для логистической и линейной регрессии
from sklearn.linear_model import LogisticRegression, LinearRegression


In [30]:
data = pd.read_csv('smoking_driking_dataset_Ver03.csv', encoding='utf-8')
print(data.head(5))

    sex  age  height  weight  waistline  sight_left  sight_right  hear_left  \
0  Male   35     170      75       90.0         1.0          1.0        1.0   
1  Male   30     180      80       89.0         0.9          1.2        1.0   
2  Male   40     165      75       91.0         1.2          1.5        1.0   
3  Male   50     175      80       91.0         1.5          1.2        1.0   
4  Male   50     165      60       80.0         1.0          1.2        1.0   

   hear_right    SBP  ...  LDL_chole  triglyceride  hemoglobin  urine_protein  \
0         1.0  120.0  ...      126.0          92.0        17.1            1.0   
1         1.0  130.0  ...      148.0         121.0        15.8            1.0   
2         1.0  120.0  ...       74.0         104.0        15.8            1.0   
3         1.0  145.0  ...      104.0         106.0        17.6            1.0   
4         1.0  138.0  ...      117.0         104.0        13.8            1.0   

   serum_creatinine  SGOT_AST  SGOT_AL

In [32]:
# Преобразуем 'sex' в числовую категорию
data['sex'] = data['sex'].map({'Male': 1, 'Female': 0}).astype(int)

# Преобразуем 'DRK_YN' в бинарные метки
data['DRK_YN'] = data['DRK_YN'].map({'Y': 1, 'N': 0}).astype(int)

# Проверка уникальных значений в целевой переменной
print("Уникальные значения в data['DRK_YN']:", data['DRK_YN'].unique())

Уникальные значения в data['DRK_YN']: [1 0]


In [34]:
columns_to_drop = [
    'SMK_stat_type_cd',  
    'sight_left', 'sight_right',  
    'hear_left', 'hear_right',  
    'serum_creatinine', 'SGOT_AST', 'SGOT_ALT', 'gamma_GTP'  
]

# Удаление столбцов
data = data.drop(columns=columns_to_drop, axis=1)

# Проверка результата
print(data.head())

   sex  age  height  weight  waistline    SBP   DBP   BLDS  tot_chole  \
0    1   35     170      75       90.0  120.0  80.0   99.0      193.0   
1    1   30     180      80       89.0  130.0  82.0  106.0      228.0   
2    1   40     165      75       91.0  120.0  70.0   98.0      136.0   
3    1   50     175      80       91.0  145.0  87.0   95.0      201.0   
4    1   50     165      60       80.0  138.0  82.0  101.0      199.0   

   HDL_chole  LDL_chole  triglyceride  hemoglobin  urine_protein  DRK_YN  
0       48.0      126.0          92.0        17.1            1.0       1  
1       55.0      148.0         121.0        15.8            1.0       0  
2       41.0       74.0         104.0        15.8            1.0       0  
3       76.0      104.0         106.0        17.6            1.0       0  
4       61.0      117.0         104.0        13.8            1.0       0  


In [36]:
# Проверка уникальных значений в целевой переменной
print("Уникальные значения в data['DRK_YN']:", data['DRK_YN'].unique())

Уникальные значения в data['DRK_YN']: [1 0]


In [38]:
# Разделение данных на признаки и целевую переменную
X = data.drop('DRK_YN', axis=1)
Y = data['DRK_YN']

In [40]:
# Масштабирование числовых признаков
scaler = StandardScaler()
numeric_features = X.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns
X[numeric_features] = scaler.fit_transform(X[numeric_features])

In [42]:
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [44]:
def regression_to_classification(x):
    return 1 if x > 0.5 else 0

In [46]:
# Обучение модели линейной регрессии
linear_model = LinearRegression()

In [48]:
linear_model.fit(X_train, Y_train)

LinearRegression()

In [50]:
linear_predictions = linear_model.predict(X_test)


In [52]:
predicted_linear = pd.Series(linear_predictions).apply(regression_to_classification)

In [54]:
# Оценка линейной регрессии
print('Оценка линейной регрессии:\n', classification_report(Y_test, predicted_linear, digits=5))


Оценка линейной регрессии:
               precision    recall  f1-score   support

           0    0.71380   0.69737   0.70549      1216
           1    0.69637   0.71284   0.70451      1184

    accuracy                        0.70500      2400
   macro avg    0.70509   0.70510   0.70500      2400
weighted avg    0.70520   0.70500   0.70501      2400



In [56]:
# Создаем объект LogisticRegression для обучения модели логистической регрессии
logistic_model = LogisticRegression()

In [58]:
logistic_model.fit(X_train, Y_train)

LogisticRegression()

In [60]:
logistic_predictions = logistic_model.predict(X_test)

In [62]:
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear'],
}


In [64]:
# Поиск оптимальных параметров с использованием GridSearchCV
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_search.fit(X_train, Y_train)


GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.1, 1, 10], 'penalty': ['l2'],
                         'solver': ['lbfgs', 'liblinear']})

In [66]:
# Получение лучших параметров и оценка модели с лучшими параметрами
best_logistic_model = grid_search.best_estimator_
logistic_predictions = best_logistic_model.predict(X_test)

In [68]:
# Оценка логистической регрессии
print('Лучшие параметры для логистической регрессии:', grid_search.best_params_)
print('Оценка логистической регрессии:\n', classification_report(Y_test, logistic_predictions, digits=5))
print('Точность логистической регрессии:', accuracy_score(Y_test, logistic_predictions))

Лучшие параметры для логистической регрессии: {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
Оценка логистической регрессии:
               precision    recall  f1-score   support

           0    0.70840   0.69326   0.70075      1216
           1    0.69174   0.70693   0.69925      1184

    accuracy                        0.70000      2400
   macro avg    0.70007   0.70009   0.70000      2400
weighted avg    0.70018   0.70000   0.70001      2400

Точность логистической регрессии: 0.7
